In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-21 16:23:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-04-21 16:23:52 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-------------------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|            Fun|It is a fun game ...|2015-01-31 00:08:00|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|    

## Drop duplicates and incomplete rows

In [6]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

5033376
5033238
5033238


## Examine the schema

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrames for desired info

In [8]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R10000VA4KCWGC|    2611860|B00HMU5VYG|     149855215|2014-01-03 00:01:00|
|R1000I8V5PQVFJ|   16426258|B004RJMUJO|     873116544|2011-01-10 00:05:00|
|R1002HOXO4X2F8|   33422100|B00QW8TYWO|     828652896|2015-01-14 00:01:00|
|R100H5JE0PTLYF|   46684956|B0060I3IJG|     216346636|2012-01-14 00:01:00|
|R100JSIEVA4OHN|   42956680|B004TTZHBK|     685224917|2011-01-29 00:07:00|
|R100MSE14DSKK2|   30495808|B00I8Q77Y0|     370420344|2014-01-03 00:03:00|
|R100NA871MOYUC|   49266280|B00FFEBXC0|     781632368|2015-01-18 00:08:00|
|R100OYY0ZUYAD2|     538377|B009G9KFQ0|     576481434|2015-01-04 00:07:00|
|R100P5IB6WZPAV|   27557243|B00MELKE0Y|     843863501|2014-01-14 00:10:00|
|R100SC3ET1HFY9|   11057625|B00Y17CHJA|     167379771|2015-01-08 00:06:00|
+--------------+---------

In [9]:
#This table will contain only unique values
products = df.select(["product_id", "product_title"])
products.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HMU5VYG|Farming Simulator...|
|B004RJMUJO|         Fruit Ninja|
|B00QW8TYWO|         Crossy Road|
|B0060I3IJG|         StumbleUpon|
|B004TTZHBK|Jamie's 20 Minute...|
|B00I8Q77Y0|Flappy Wings (not...|
|B00FFEBXC0|           U.S. Bank|
|B009G9KFQ0|WeatherBug - Free...|
|B00MELKE0Y|         Back to Bed|
|B00Y17CHJA|             Youtube|
+----------+--------------------+
only showing top 10 rows



In [10]:
# Drop duplicates due to some ids and names repeating
products= products.dropDuplicates()
products.count()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00KQ1XYFQ|   My Verizon Mobile|
|B004XJDALK|Learn Math via vi...|
|B00ITR6WLW|How to Draw: Eque...|
|B00I2N3M4I|     Eldoron Defense|
|B00D2HWE5M|Slender Man Chapt...|
+----------+--------------------+
only showing top 5 rows



In [11]:
customers = df.select(["customer_id"])
customers.show(10)

+-----------+
|customer_id|
+-----------+
|    2611860|
|   16426258|
|   33422100|
|   46684956|
|   42956680|
|   30495808|
|   49266280|
|     538377|
|   27557243|
|   11057625|
+-----------+
only showing top 10 rows



In [12]:
customers = customers.groupby(['customer_id']).count()
customers.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   24501591|   39|
|    1517738|    5|
|   38348313|   13|
|   47736539|    2|
|   18782075|    1|
|   33007868|    2|
|   24984681|    1|
|   27501301|    6|
|   49502298|    3|
|   21998610|   20|
+-----------+-----+
only showing top 10 rows



In [13]:
customers = customers.withColumnRenamed("count", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24501591|            39|
|    1517738|             5|
|   38348313|            13|
|   47736539|             2|
|   18782075|             1|
+-----------+--------------+
only showing top 5 rows



In [14]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10000VA4KCWGC|          5|            6|         15|   N|
|R1000I8V5PQVFJ|          5|            1|          1|   N|
|R1002HOXO4X2F8|          5|            0|          0|   N|
|R100H5JE0PTLYF|          5|            0|          0|   N|
|R100JSIEVA4OHN|          5|            0|          0|   N|
|R100MSE14DSKK2|          4|            0|          0|   N|
|R100NA871MOYUC|          1|            0|          0|   N|
|R100OYY0ZUYAD2|          5|            0|          0|   N|
|R100P5IB6WZPAV|          5|            0|          0|   N|
|R100SC3ET1HFY9|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Write DataFrame to RDS

In [15]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<end point>:<port>/<data_base>"
config = {"user":"user_name",
          "password": "<password here",
          "driver":"org.postgresql.Driver"}

## Write DataFrame to RDS

In [16]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)